TWITTER SENTIMENT ANALYSIS

MODEL BUILDING

IMPORTING NECESSARY LIBRARIES

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

LOADING THE DATASETS

In [3]:
tweets = pd.read_csv(r'C:\Projects\Twitter sentiment analysis\cleaned_tweets.csv')
ht_p = pd.read_csv(r'C:\Projects\Twitter sentiment analysis\positive_hashtags.csv')
ht_n = pd.read_csv(r'C:\Projects\Twitter sentiment analysis\negative_hashtags.csv')

In [4]:
# Filter and create a DataFrame with rows containing missing values in the "clean_tweets" column
missing_tweets_df = tweets[tweets["clean_tweets"].isnull()]

missing_tweets_df.head()

,target,id,date,flag,user,text,emoji,preprocessed,clean_tweets
3997,0,1468720797,Tue Apr 07 03:21:48 PDT 2009,NO_QUERY,mohdnajwan,what to do,False,[],NaN
4233,0,1468771833,Tue Apr 07 03:39:06 PDT 2009,NO_QUERY,goldieloxx,just,False,[],NaN
5284,0,1469038754,Tue Apr 07 04:57:00 PDT 2009,NO_QUERY,warisara,and here http://twitpic.com/2yaxf,False,[],NaN
18950,0,1556733475,Sun Apr 19 00:05:23 PDT 2009,NO_QUERY,LouiseRedknapp,up again,False,[],NaN
27140,0,1559064681,Sun Apr 19 10:09:43 PDT 2009,NO_QUERY,horriddeath,and it did,False,[],NaN


In [5]:
# Remove rows with missing values in the "clean_tweets" column
tweets = tweets.dropna(subset=["clean_tweets"])

MODEL PART

In [6]:
# Splitting data into features: tweets and labels
X = tweets["clean_tweets"]
y = tweets["target"]

In [7]:
# Train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=10000)

In [9]:
# Fit the vectorizer on the training data and transform the tweets into TF-IDF features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

HASHTAGS

In [12]:
# Create dictionaries for positive and negative hashtag frequencies
positive_hashtag_freq = dict(zip(ht_p["Hashtag"], ht_p["Count"]))
negative_hashtag_freq = dict(zip(ht_p["Hashtag"], ht_p["Count"]))

In [14]:
# Create arrays for positive and negative hashtag frequencies for each tweet
X_train_hashtags = []
X_test_hashtags = []

for i in [X_train, X_test]:
    hashtags_freq = []
    for j in i:
        pos_freq = sum(positive_hashtag_freq.get(hashtag, 0) for hashtag in j.split())
        neg_freq = sum(negative_hashtag_freq.get(hashtag, 0) for hashtag in j.split())
        hashtags_freq.append([pos_freq, neg_freq])
    if i is X_train:
        X_train_hashtags = hashtags_freq
    else:
        X_test_hashtags = hashtags_freq


In [17]:
# Convert the hashtag frequency lists to NumPy arrays
X_train_hashtags = np.array(X_train_hashtags)
X_test_hashtags = np.array(X_test_hashtags)

# Concatenate the TF-IDF features and hashtag frequency features
from scipy.sparse import hstack

X_train_combined = hstack((X_train_tfidf, X_train_hashtags))
X_test_combined = hstack((X_test_tfidf, X_test_hashtags))

model = LogisticRegression(max_iter=10000)

# Fit the model with combined features
model.fit(X_train_combined, y_train)

# Predict sentiment on the test data
y_pred = model.predict(X_test_combined)

# Evaluate the model's performance
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", class_report)

Accuracy: 0.771594326116991
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.75      0.77    159614
           4       0.76      0.79      0.78    160308

    accuracy                           0.77    319922
   macro avg       0.77      0.77      0.77    319922
weighted avg       0.77      0.77      0.77    319922

